In [1]:
#加载数据，数据存放在all_records变量中

import pandas as pd
import os
path='login'
files=os.listdir(path)
all_records=pd.DataFrame()
for file in files:
    f=pd.read_csv(path+'/'+file)
    #我这里必须要加这个赋值语句，可以修改
    all_records=all_records.append(f)
all_records.head()

,Unnamed: 0,destination_ip,destination_port,info,password,protocol,record_time,source_ip,source_port,user
0,0,10.49.22.87,21,530 Logout.,f379eaf3c831b04de153469d1bec345e,ftp,2019-05-30 23:51:46.734,10.49.212.206,35910,lss6
1,1,10.49.22.87,21,530 Login authentication failed,f379eaf3c831b04de153469d1bec345e,ftp,2019-05-30 23:51:46.734,10.49.212.206,35910,lss6
2,2,10.24.249.104,3306,mysql login success,6c570961c863501d1af3346af287b801,mysql,2019-05-30 23:51:46.921,10.56.92.187,56173,lss6
3,3,10.24.249.104,3306,mysql login success,0b470c0ef029a53a2111561dcf2a642b,mysql,2019-05-30 23:51:47.427,10.56.92.187,56174,lss6
4,4,10.49.148.131,3306,mysql login success,1db2720439a620e362a8aae227fe7493,mysql,2019-05-30 23:51:47.791,10.49.148.130,55799,uyhrnf


In [16]:
#统计登录info

tmp = all_records.groupby(['info'])
infos = []
i = 0
for name, group in  tmp:
    infos.append(name)
    group.to_csv("login_code" + "/" + str(i)+".csv", index=False)
    i += 1
tmp_frame = pd.DataFrame()
tmp_frame['info'] = infos
#tmp_frame.to_csv("login_info.csv", index=False)  #这里生成csv
print(infos)

['230 Anonymous access granted, restrictions apply', '230 Login successful.', '230 User logged in, proceed.', '331 User root OK. Password required', '530 Login authentication failed', '530 Login or password incorrect!', '530 Logout.', '530 Sorry, no ANONYMOUS access allowed.', '530 û��Ȩ��', 'STATUS_LOGON_FAILURE', 'STATUS_LOGON_SUCCESS', 'Unknow command AUTH ', 'authentication successed', 'authorization success', 'internal server error', 'mysql login success', '�\u07b7���½�������û�POP����δ��ͨ��authorization failed']


In [24]:
#删选服务器

tmp = all_records.groupby(['destination_ip'])
ips = []
times = []
protocols = []
src_ips = []
for name, group in tmp:
    ips.append(name)
    times.append(group.shape[0])
    this_protocol = []
    tmp2 = group.groupby(['protocol'])
    for n, g in tmp2:
        this_protocol.append(n)
    protocols.append(this_protocol)
    src_ip = []
    tmp3 = group.groupby(['source_ip'])
    for n, g in tmp3:
        src_ip.append(n)
    src_ips.append(len(src_ip))
    
tmp_frame = pd.DataFrame()
tmp_frame['ip'] = ips
tmp_frame['time'] = times
tmp_frame['protocol'] = protocols
tmp_frame['source_ip'] = src_ips
tmp_frame.to_csv("login_code/dst_ip.csv", index = False)

In [5]:
#删选指定dst ip的所有记录

ip = ["10.49.22.87"]    #要查询的IP
query = pd.DataFrame()
query = all_records[all_records['destination_ip'].isin(ip)]
#query.to_csv("dst"+ip[0] + ".csv", index = False)

In [7]:
#以下代码基于上面的查询，现在是查询与该IP通信的所有用户

tmp = query.groupby(['info'])
for name, group in tmp:
    print(name)
    print(group.shape)

331 User root OK. Password required
(8582, 10)
530 Login authentication failed
(3526265, 10)
530 Logout.
(2678144, 10)


In [4]:
#登录失败IP

data = pd.read_csv("login_code/4.csv")
tmp = data.groupby(['source_ip', 'user'])
for name, group in tmp:
    name = list(name)
    #group.to_csv("login_failed/" + name[0] + name[1] + '.csv', index = False)
    print(group.shape)
    

(3526265, 10)


In [29]:
print(data.head())

   Unnamed: 0 destination_ip  destination_port  \
0           1    10.49.22.87                21   
1           6    10.49.22.87                21   
2           7    10.49.22.87                21   
3          10    10.49.22.87                21   
4          11    10.49.22.87                21   

                              info                          password protocol  \
0  530 Login authentication failed  f379eaf3c831b04de153469d1bec345e      ftp   
1  530 Login authentication failed  96e79218965eb72c92a549dd5a330112      ftp   
2  530 Login authentication failed  96e79218965eb72c92a549dd5a330112      ftp   
3  530 Login authentication failed  96e79218965eb72c92a549dd5a330112      ftp   
4  530 Login authentication failed  9d4568c009d203ab10e33ea9953a0264      ftp   

               record_time      source_ip  source_port  user  
0  2019-05-30 23:51:46.734  10.49.212.206        35910  lss6  
1  2019-05-30 23:51:48.190  10.49.212.206        36062  lss6  
2  2019-05-30 23:51:48.

In [30]:
#按协议切login日志

tmp = all_records.groupby(['protocol'])
for name, group in tmp:
    group.to_csv("login_protocol/" + name + ".csv", index = False)

In [3]:
#查询IP与用户是否对应

src_ips = []
users = []

tmp = all_records.groupby(['source_ip', 'user'])
for name, group in tmp:
    name = list(name)
    src_ips.append(name[0])
    users.append(name[1])
    
tmp_frame = pd.DataFrame()
tmp_frame['ip'] = src_ips
tmp_frame['user'] = users
tmp_frame.to_csv("user_ip.csv", index = False)